In [37]:
#importing
import numpy as np
import pandas as pd
import multiprocessing
import math 
from numpy import mean
from statistics import variance
from numpy import std
from pandas import read_csv
from sklearn.impute import KNNImputer
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error as mae
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
import warnings

In [38]:
#warnings.filterwarnings('ignore')

In [3]:
multiprocessing.cpu_count()

8

In [4]:
data_wd = "C:/Users/Lenovo/Desktop/git/STAT4011_Project1/data/Lianjia_transformed.csv"
df = pd.read_csv(data_wd)

In [5]:
X = df.drop(columns=['price'])
y = df['price']
Xtrain,Xtest,ytrain,ytest=train_test_split(X,y,test_size=0.2,random_state=4011)

# Define a model Assess Function

In [8]:
def model_assess(tuned_model,X_train,y_train,X_test,y_test):
    y_train_pred=tuned_model.predict(X_train)
    y_test_pred=tuned_model.predict(X_test)
    train_R2 = tuned_model.score(X_train, y_train)
    test_R2 = tuned_model.score(X_test, y_test)
    train_RMSE=round(math.sqrt(mean_squared_error(y_train_pred,y_train)),4)
    test_RMSE=round(math.sqrt(mean_squared_error(y_test_pred,y_test)),4)
    train_MAE=round(mae(y_train_pred,ytrain),4)
    test_MAE=round(mae(y_test_pred,ytest),4)
    return([train_R2,test_R2,train_RMSE,test_RMSE,train_MAE,test_MAE])

# OLS

In [9]:
model_ols = LinearRegression().fit(Xtrain, ytrain)

In [10]:
ols_result=model_assess(model_ols,Xtrain,ytrain,Xtest,ytest)

# LASSO

In [11]:
# step 1: examine the model
model_lasso = LassoCV(cv=5, random_state=4011,n_jobs=-2,alphas=np.logspace(-5,-1,100),tol=1e-2).fit(Xtrain, ytrain)

In [12]:
# step 2: examine if the selected hyperparameter lies within boundary
model_lasso.alpha_ 

0.0038535285937105314

In [13]:
# step 3: model assessment
lasso_result=model_assess(model_lasso,Xtrain,ytrain,Xtest,ytest)

In [14]:
# step 4: examine if coefficients are shrinked towards 0; this is only for lasso and ridge
print(list(zip(model_lasso.coef_, X))) 

[(6.746396362533325, 'total_floor'), (26.433482136701418, 'area_size'), (3.7420521258645536, 'elevator'), (-0.42096604583414676, 'date_on_market'), (0.34081364442938944, 'last_transact_date'), (1.1199652772022928, 'bedrm_no'), (0.9128766955498774, 'parlour_no'), (0.048710482182980686, 'kitchen_no'), (3.94072790244851, 'toilet_no'), (-3.904172401406128, 'stair'), (2.142749096850644, 'unit'), (4.636422005474458, 'proportion'), (0.8743437420303899, 'mid_floor'), (0.08073072934462963, 'higher_floor'), (-0.8475476505669389, 'duplex_config'), (0.46740503400994804, 'levelling_config'), (0.2150233561903785, 'split_level_config'), (0.19744061750489922, 'diepin'), (-2.1963970291601607, 'talou'), (0.25537760298219936, 'pingfang'), (0.2947684186077513, 'bantajiehe'), (1.3968058560207328, 'banlou'), (2.153574876872985, 'dudong'), (-1.3155647517181457, 'lianpai'), (0.44731355258225675, 'south'), (-0.1443868466026778, 'west'), (-0.17237575966748908, 'north'), (-0.4619997462640114, 'northeast'), (0.23

# RIDGE

In [22]:
model_ridge = RidgeCV(cv=5,alphas=np.logspace(-1,3,100)).fit(Xtrain, ytrain)
model_ridge.alpha_ 

81.11308307896873

In [23]:
ridge_result=model_assess(model_ridge,Xtrain,ytrain,Xtest,ytest)

In [24]:
#examine if coefficients are shrinked 
print(list(zip(model_ridge.coef_, X))) 

[(6.706065953939077, 'total_floor'), (25.97101465070193, 'area_size'), (3.7343712781167637, 'elevator'), (-0.4083998722356938, 'date_on_market'), (0.360552376120507, 'last_transact_date'), (1.292567365242712, 'bedrm_no'), (0.9637350515243888, 'parlour_no'), (0.048777130026567236, 'kitchen_no'), (4.082070884338945, 'toilet_no'), (-3.816967999201759, 'stair'), (2.0351392702175866, 'unit'), (4.606564294168615, 'proportion'), (0.858490799554046, 'mid_floor'), (0.06009602990876299, 'higher_floor'), (-0.8212047976799733, 'duplex_config'), (0.45694061694912474, 'levelling_config'), (0.21755780014733972, 'split_level_config'), (0.3012927271798949, 'diepin'), (-2.0723541450762886, 'talou'), (0.26495266367398296, 'pingfang'), (0.3470947386969166, 'bantajiehe'), (1.490416283581285, 'banlou'), (2.3140648088912186, 'dudong'), (-1.1473689306726822, 'lianpai'), (0.4612726397069366, 'south'), (-0.13667745363910186, 'west'), (-0.18127108084585633, 'north'), (-0.4587489845465758, 'northeast'), (0.239388

# Elastic Net

In [62]:
model_elanet = ElasticNetCV(cv=5,l1_ratio=[0.5,0.7,0.8,0.9,0.95,1],alphas=np.logspace(-10,-1,20),n_jobs=-2,tol=1e-1,max_iter=1e5).fit(Xtrain, ytrain)

In [63]:
model_elanet.alpha_

5.455594781168514e-06

In [64]:
model_elanet.l1_ratio_ 

1.0

In [65]:
elanet_result=model_assess(model_elanet,Xtrain,ytrain,Xtest,ytest)

# SVM

In [ ]:
param_grid_svr = [
    {'kernel': ['poly'], 'degree': [1,2,3,4,5], 'tol': [1e-3,1e-5,1e-7], 
     'C': [0.1,1,10,100], 'epsilon': [1e-3,1e-2,0.1,1]},
    {'kernel': ['sigmoid'], 'tol': [1e-3,1e-5,1e-7], 'C': [0.1,1,10,100], 
     'epsilon': [1e-3,1e-2,0.1,1]},
    {'kernel': ['linear','rbf'], 'tol': [1e-3,1e-5,1e-7], 'C': [0.1,1,10,100], 'epsilon': [1e-3,1e-2,0.1,1]}
]

select_model_svr = GridSearchCV(SVR(),param_grid_svr, n_jobs=-2, cv=5,scoring='neg_mean_squared_error',
return_train_score=True)
select_model_svr.fit(Xtrain, ytrain)

model_svr = select_model_svr.best_estimator_
select_model_svr.best_params_

In [ ]:
svr_result = model_assess(model_svr,X_train,y_train,X_test,y_test)

# Decision Tree

In [ ]:
param_grid_tree = {'max_depth':[10,15,20,25,30], 'min_samples_split': [1,2,3,4], 
 'min_samples_leaf': [1,2,3,4,5], 'max_features': [50,100,150], 'max_leaf_nodes': [250,300,350,400,450,500]}

tree = DecisionTreeRegressor(random_state=4011)
select_model_tree = GridSearchCV(tree, param_grid_tree, n_jobs=-2, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
select_model_tree.fit(Xtrain, ytrain)

model_tree_reg = select_model_tree.best_estimator_
select_model_tree.best_params_

In [ ]:
tree_reg_result = model_assess(model_tree_reg,X_train,y_train,X_test,y_test)

# Random Forest

In [ ]:
param_grid_rfr = {'n_estimators': [26,30, 32, 34,38,42], 'max_features': [50,100,200,300]},

forest_reg = RandomForestRegressor(random_state=4011)
select_model_rfr = GridSearchCV(forest_reg, param_grid_rfr, n_jobs=-2, cv=5,
scoring='neg_mean_squared_error',
return_train_score=True)
select_model_rfr.fit(Xtrain, ytrain)

model_rfr = select_model_rfr.best_estimator_
select_model_rfr.best_params_

In [ ]:
rfr_result = model_assess(model_rfr,X_train,y_train,X_test,y_test)

# Save All Results

In [56]:
all_results=np.vstack([ols_result,lasso_result,ridge_result,elanet_result])

In [57]:
all_results=pd.DataFrame(all_results)
all_results.columns=["train_R2","test_R2","train_RMSE","test_RMSE","train_MAE","test_MAE"]
all_results.index=["ols","lasso","ridge","elastic_net"]

In [58]:
all_results

train_R2   test_R2  train_RMSE  test_RMSE  train_MAE  test_MAE
ols          0.792032  0.773265     19.4346    21.9289    12.0421   12.5958
lasso        0.792025  0.773246     19.4349    21.9298    12.0369   12.5899
ridge        0.791957  0.772686     19.4380    21.9569    12.0241   12.5830
elastic_net  0.792032  0.773264     19.4346    21.9289    12.0421   12.5958